In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df_cleancoll = pd.read_csv('cleanVIF_loan_test4.csv')

In [15]:
pd.set_option('display.max_columns', None)

In [16]:
import re

regex = re.compile(r"\[|\]|<", re.IGNORECASE)

df_cleancoll.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in df_cleancoll.columns.values]

In [17]:
df_cleancoll.head()

,sub_grade_A1,sub_grade_A2,sub_grade_A3,sub_grade_A4,sub_grade_A5,sub_grade_B1,sub_grade_B2,sub_grade_B3,sub_grade_B4,sub_grade_B5,sub_grade_C1,sub_grade_C2,sub_grade_C4,sub_grade_C5,sub_grade_D1,sub_grade_D2,sub_grade_D3,sub_grade_D4,sub_grade_D5,sub_grade_E1,sub_grade_E2,sub_grade_E3,sub_grade_E4,sub_grade_E5,sub_grade_F1,sub_grade_F2,sub_grade_F3,sub_grade_F4,sub_grade_F5,sub_grade_G1,sub_grade_G2,sub_grade_G3,sub_grade_G4,sub_grade_G5,emp_length_10+ years,emp_length_5 years,emp_length_6 years,emp_length_7 years,emp_length__ 1 year,issue_d_2012-08-01,issue_d_2012-09-01,issue_d_2012-10-01,issue_d_2012-11-01,issue_d_2012-12-01,issue_d_2013-01-01,issue_d_2013-02-01,issue_d_2013-03-01,issue_d_2013-04-01,issue_d_2013-05-01,issue_d_2013-06-01,issue_d_2013-07-01,issue_d_2013-08-01,issue_d_2013-09-01,issue_d_2013-10-01,issue_d_2013-11-01,issue_d_2013-12-01,issue_d_2014-01-01,issue_d_2014-02-01,issue_d_2014-03-01,issue_d_2014-04-01,issue_d_2014-05-01,issue_d_2014-06-01,issue_d_2014-07-01,issue_d_2014-08-01,issue_d_2014-09-01,issue_d_2014-10-01,issue_d_2015-01-01,issue_d_2015-02-01,issue_d_2015-03-01,issue_d_2015-04-01,issue_d_2015-05-01,issue_d_2015-06-01,issue_d_2015-07-01,issue_d_2015-08-01,issue_d_2015-09-01,issue_d_2015-10-01,issue_d_2015-11-01,issue_d_2015-12-01,purpose_car,purpose_credit_card,purpose_home_improvement,purpose_house,purpose_medical,purpose_moving,purpose_other,purpose_small_business,purpose_wedding,addr_state_AL,addr_state_CA,addr_state_CO,addr_state_CT,addr_state_DC,addr_state_FL,addr_state_GA,addr_state_HI,addr_state_IL,addr_state_ME,addr_state_MS,addr_state_ND,addr_state_NE,addr_state_NH,addr_state_NV,addr_state_NY,addr_state_SC,addr_state_TX,addr_state_VA,addr_state_VT,addr_state_WV,earliest_cr_line_2007-03-01,last_pymnt_d_2015-09-01,last_pymnt_d_2015-10-01,last_pymnt_d_2015-11-01,last_pymnt_d_2015-12-01,next_pymnt_d_2015-10-01,next_pymnt_d_2015-11-01,next_pymnt_d_2015-12-01,next_pymnt_d_2016-01-01,last_credit_pull_d_2013-03-01,last_credit_pull_d_2013-04-01,last_credit_pull_d_2013-05-01,last_credit_pull_d_2013-06-01,last_credit_pull_d_2013-07-01,last_credit_pull_d_2013-08-01,last_credit_pull_d_2013-09-01,last_credit_pull_d_2013-10-01,last_credit_pull_d_2013-11-01,last_credit_pull_d_2013-12-01,last_credit_pull_d_2014-01-01,last_credit_pull_d_2014-02-01,last_credit_pull_d_2014-03-01,last_credit_pull_d_2014-04-01,last_credit_pull_d_2014-05-01,last_credit_pull_d_2014-06-01,last_credit_pull_d_2014-07-01,last_credit_pull_d_2014-08-01,last_credit_pull_d_2014-09-01,last_credit_pull_d_2014-10-01,last_credit_pull_d_2014-11-01,last_credit_pull_d_2014-12-01,annual_inc,dti,inq_last_6mths,mths_since_last_record,pub_rec,revol_bal,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,flag_bad_loan
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,55000.0,22.87,0.0,0.0,0.0,36638.0,0.0,885.46,0.0,0.0,114834.0,59900.0,0
1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26000.0,25.12,0.0,0.0,0.0,7967.0,0.0,333.14,0.0,0.0,14123.0,15100.0,0
2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,105000.0,14.05,1.0,0.0,0.0,13168.0,0.0,368.45,0.0,0.0,267646.0,61100.0,0
3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,

## 5. MODELLING

#### Train Test Split

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
X = df_cleancoll.drop('flag_bad_loan',axis=1)
y = df_cleancoll['flag_bad_loan']

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=101)

In [21]:
y_test.head()

275869    0
713700    0
152931    0
383013    1
411552    0
Name: flag_bad_loan, dtype: int64

#### XGBoost

Unmodified Data

In [22]:
from xgboost import XGBClassifier
xgboost = XGBClassifier(learning_rate = 0.01, n_estimators = 1000, max_depth = 4, n_jobs = -1)

In [23]:
xgboost.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.01, max_delta_step=0, max_depth=4,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=1000, n_jobs=-1, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
              validate_parameters=False, verbosity=None)

In [25]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_train, xgboost.predict(X_train)))

              precision    recall  f1-score   support

           0       0.97      1.00      0.99    538461
           1       0.96      0.54      0.69     33153

    accuracy                           0.97    571614
   macro avg       0.97      0.77      0.84    571614
weighted avg       0.97      0.97      0.97    571614



In [29]:
print(classification_report(y_test,xgboost.predict(X_test)))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98    230233
           1       0.96      0.54      0.69     14745

    accuracy                           0.97    244978
   macro avg       0.97      0.77      0.84    244978
weighted avg       0.97      0.97      0.97    244978



Modified Data - SMOTE

In [30]:
from imblearn.over_sampling import SMOTE

# Resample the minority class. You can change the strategy to 'auto' if you are not sure.
sm = SMOTE(sampling_strategy='minority')

oversampled_trainX, oversampled_trainY = sm.fit_sample(X_train, y_train)
oversampled_train = pd.concat([pd.DataFrame(oversampled_trainY), pd.DataFrame(oversampled_trainX)], axis =1)

In [31]:
xgboost.fit(oversampled_trainX,oversampled_trainY)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.01, max_delta_step=0, max_depth=4,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=1000, n_jobs=-1, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
              validate_parameters=False, verbosity=None)

In [32]:
print(classification_report(oversampled_trainY, xgboost.predict(oversampled_trainX))) 

              precision    recall  f1-score   support

           0       0.94      0.98      0.96    538461
           1       0.98      0.94      0.96    538461

    accuracy                           0.96   1076922
   macro avg       0.96      0.96      0.96   1076922
weighted avg       0.96      0.96      0.96   1076922



In [33]:
print(classification_report(y_test, xgboost.predict(X_test)))

              precision    recall  f1-score   support

           0       0.97      0.98      0.97    230233
           1       0.65      0.47      0.54     14745

    accuracy                           0.95    244978
   macro avg       0.81      0.73      0.76    244978
weighted avg       0.95      0.95      0.95    244978



XGBoost with PCA

In [37]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [38]:
def try_w_pca(df, train_index, test_index):
    scaler = StandardScaler()
    target = df['flag_bad_loan']
    df = df.drop('flag_bad_loan', axis = 1)
    df = pd.DataFrame(scaler.fit_transform(df), columns = df.columns)
    for pc in range(2, 11):
        pca = PCA(n_components = pc)
        df_pca =pd.DataFrame(pca.fit_transform(df))
        X_train = df_pca.iloc[train_index]
        X_test = df_pca.iloc[test_index]
        y_train = target.iloc[train_index]
        y_test = target.iloc[test_index]
        xgboost.fit(X_train, y_train)
        print('pc = {} and train'.format(pc))
        print(classification_report(y_train, xgboost.predict(X_train)))
        print('pc = {} and test'.format(pc))
        print(classification_report(y_test, xgboost.predict(X_test)))

In [39]:
try_w_pca(df_cleancoll, X_train.index, X_test.index)

pc = 2 and train
              precision    recall  f1-score   support

           0       0.95      1.00      0.97    538461
           1       0.69      0.09      0.15     33153

    accuracy                           0.94    571614
   macro avg       0.82      0.54      0.56    571614
weighted avg       0.93      0.94      0.92    571614

pc = 2 and test
              precision    recall  f1-score   support

           0       0.94      1.00      0.97    230233
           1       0.70      0.09      0.15     14745

    accuracy                           0.94    244978
   macro avg       0.82      0.54      0.56    244978
weighted avg       0.93      0.94      0.92    244978

pc = 3 and train
              precision    recall  f1-score   support

           0       0.95      1.00      0.97    538461
           1       0.98      0.14      0.25     33153

    accuracy                           0.95    571614
   macro avg       0.97      0.57      0.61    571614
weighted avg       0.95 